<a href="https://colab.research.google.com/github/tnstanHCMUS/TQHDL/blob/main/Profilling_Pre-processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Đồ án lý thuyết môn trực quan hoá: Netflix dataset</center>

# Thông tin sinh viên

- Nhóm 10:
    + 22127163 - Trần Đan Huy
    + 22127290 - Nguyễn Thị Thu Ngân
    + 22127029 - Lê Nguyễn Gia Bảo
    + 22127026 - Ôn Gia Bảo

# Import

In [ ]:
import pandas as pd
import numpy as np
import os

# Tải csv lên hệ thống


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving netflix_titles.csv to netflix_titles.csv


# Đọc dữ liệu

In [ ]:
# Đọc dữ liệu bằng pandas
#tắt chức năng tự chuyển giá trị trống thành NaN của pandas
df = pd.read_csv('netflix_titles.csv', encoding='utf-8-sig', keep_default_na=False)
#df.head()

#Data profilling

###1. Overview

In [ ]:
# Tổng số dòng
total_rows = len(df)

# Hàm phân tích chất lượng cột
def column_quality(col):
    total = total_rows
    nulls = df[col].isnull().sum()
    missing = df[col].apply(lambda x: isinstance(x, str) and x.strip() == '').sum()
    actual = total - nulls - missing
    completeness = actual / total if total > 0 else 0
    cardinality = df[col].nunique(dropna=True)
    uniqueness = cardinality / actual if actual > 0 else 0
    distinctness = cardinality / total if total > 0 else 0
    dtype = df[col].dtype

    return pd.Series({
        'Data Type': dtype,
        'Total': total,
        'NULL': nulls,
        'Missing': missing,
        'Actual': actual,
        'Completeness (%)': f"{completeness * 100:.3f}%",
        'Cardinality': cardinality,
        'Uniqueness (%)': f"{uniqueness * 100:.3f}%",
        'Distinctness (%)': f"{distinctness * 100:.3f}%"
    })

# Áp dụng cho toàn bộ cột
quality_df = df.columns.to_series().apply(column_quality)

# Định dạng lại bảng
quality_df = quality_df.reset_index().rename(columns={'index': 'Column'})
quality_df.index += 1  # STT bắt đầu từ 1

quality_df

,Column,Data Type,Total,NULL,Missing,Actual,Completeness (%),Cardinality,Uniqueness (%),Distinctness (%)
1,show_id,object,8807,0,0,8807,100.000%,8807,100.000%,100.000%
2,type,object,8807,0,0,8807,100.000%,2,0.023%,0.023%
3,title,object,8807,0,0,8807,100.000%,8807,100.000%,100.000%
4,director,object,8807,0,2634,6173,70.092%,4529,73.368%,51.425%
5,cast,object,8807,0,825,7982,90.632%,7693,96.379%,87.351%
6,country,object,8807,0,831,7976,90.564%,749,9.391%,8.505%
7,date_added,object,8807,0,10,8797,99.886%,1768,20.098%,20.075%
8,release_year,int64,8807,0,0,8807,100.000%,74,0.840%,0.840%
9,rating,object,8807,0,4,8803,99.955%,18,0.204%,0.204%
10,duration,object,8807,0,3,8804,99.966%,221,2.510%,2.509%


Vì một số thuộc tính có Completeness tương đối thấp (~70%) nên không tiền xử lý dự liệu bằng cách xoá các dòng dữ liệu trống, thay vào đó, nhóm quyết định lấp đầy các chỗ trống với kiểu dữ liệu phù hợp


###2. Details

In [ ]:
def analyze_column(df, column):
    from termcolor import colored
    from IPython.display import Markdown, display
    import pandas as pd

    col_data = df[column]
    total = len(col_data)
    actual = col_data.notnull().sum()
    missing = col_data.isnull().sum()
    completeness = round(actual / total * 100, 2)
    cardinality = col_data.nunique()
    uniqueness = round((col_data.value_counts() == 1).sum() / actual * 100, 2)
    distinctness = round(cardinality / actual * 100, 2)
    min_len = col_data.dropna().astype(str).apply(len).min()
    max_len = col_data.dropna().astype(str).apply(len).max()
    data_type = col_data.dropna().map(type).mode()[0].__name__ if actual > 0 else 'unknown'
    sample_val = str(col_data.dropna().astype(str).iloc[0]) if actual > 0 else ''
    field_format = ''.join(['n' if c.isdigit() else 'x' for c in sample_val])
    top5 = col_data.value_counts().head(5)

    def print_section(title):
        display(Markdown(f"### <span style='color:#2e6cbb'>{title}</span>"))

    def print_kv(label, value):
        print(f"{label:<32}: {value}")
        print("-" * 50)

    print_section(f"🔎 INPUT METADATA — {column.upper()}")
    print_kv("Field Name", column)
    print_kv("Field Data Type", data_type.upper())
    print_kv("Field Format", field_format)
    print_kv("Field Length", max_len)

    print_section("📊 DATA PROFILING SUMMARY STATISTICS")
    print_kv("NULL", 0)
    print_kv("Missing", missing)
    print_kv("Actual", actual)
    print_kv("Completeness", f"{completeness}%")
    print_kv("Cardinality", cardinality)
    print_kv("Uniqueness", f"{uniqueness}%")
    print_kv("Distinctness", f"{distinctness}%")

    print_section("📐 DATA PROFILING ADDITIONAL STATISTICS")
    print_kv("Field Data Types", 1)
    print_kv("Field Length (MIN)", min_len)
    print_kv("Field Length (MAX)", max_len)
    if pd.api.types.is_numeric_dtype(col_data.dropna()):
        print_kv("Field Value (MIN)", col_data.min())
        print_kv("Field Value (MAX)", col_data.max())

    print_section(f"⭐ {column.upper()} — TOP 5 VALUES")
    print(f"{'Value':<30}{'Count':>10}{'Percentage':>12}")
    print("-" * 55)
    for val, count in top5.items():
        percent = round(count / total * 100, 2)
        val_str = str(val)[:30]
        print(f"{val_str:<30}{count:>10}{percent:>12}%")


####2.1. show_id

####2.1. show_id

In [ ]:
#viết hàm thực thi cho thuộc tính show_id của df
analyze_column(df, 'show_id')


### <span style='color:#2e6cbb'>🔎 INPUT METADATA — SHOW_ID</span>

Field Name                      : show_id
--------------------------------------------------
Field Data Type                 : STR
--------------------------------------------------
Field Format                    : xn
--------------------------------------------------
Field Length                    : 5
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

NULL                            : 0
--------------------------------------------------
Missing                         : 0
--------------------------------------------------
Actual                          : 8807
--------------------------------------------------
Completeness                    : 100.0%
--------------------------------------------------
Cardinality                     : 8807
--------------------------------------------------
Uniqueness                      : 100.0%
--------------------------------------------------
Distinctness                    : 100.0%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types                : 1
--------------------------------------------------
Field Length (MIN)              : 2
--------------------------------------------------
Field Length (MAX)              : 5
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ SHOW_ID — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
s8807                                  1        0.01%
s1                                     1        0.01%
s2                                     1        0.01%
s3                                     1        0.01%
s4                                     1        0.01%


####2.2. type

In [ ]:
analyze_column(df, 'type')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — TYPE</span>

Field Name                      : type
--------------------------------------------------
Field Data Type                 : STR
--------------------------------------------------
Field Format                    : xxxxx
--------------------------------------------------
Field Length                    : 7
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

NULL                            : 0
--------------------------------------------------
Missing                         : 0
--------------------------------------------------
Actual                          : 8807
--------------------------------------------------
Completeness                    : 100.0%
--------------------------------------------------
Cardinality                     : 2
--------------------------------------------------
Uniqueness                      : 0.0%
--------------------------------------------------
Distinctness                    : 0.02%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types                : 1
--------------------------------------------------
Field Length (MIN)              : 5
--------------------------------------------------
Field Length (MAX)              : 7
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ TYPE — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
Movie                               6131       69.62%
TV Show                             2676       30.38%


###2.3. title

In [ ]:
analyze_column(df, 'title')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — TITLE</span>

Field Name                      : title
--------------------------------------------------
Field Data Type                 : STR
--------------------------------------------------
Field Format                    : xxxxxxxxxxxxxxxxxxxx
--------------------------------------------------
Field Length                    : 104
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

NULL                            : 0
--------------------------------------------------
Missing                         : 0
--------------------------------------------------
Actual                          : 8807
--------------------------------------------------
Completeness                    : 100.0%
--------------------------------------------------
Cardinality                     : 8807
--------------------------------------------------
Uniqueness                      : 100.0%
--------------------------------------------------
Distinctness                    : 100.0%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types                : 1
--------------------------------------------------
Field Length (MIN)              : 1
--------------------------------------------------
Field Length (MAX)              : 104
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ TITLE — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
Zubaan                                 1        0.01%
Dick Johnson Is Dead                   1        0.01%
Blood & Water                          1        0.01%
Ganglands                              1        0.01%
Jailbirds New Orleans                  1        0.01%


###2.4. director

In [ ]:
analyze_column(df, 'director')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — DIRECTOR</span>

Field Name                      : director
--------------------------------------------------
Field Data Type                 : STR
--------------------------------------------------
Field Format                    : xxxxxxxxxxxxxxx
--------------------------------------------------
Field Length                    : 208
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

NULL                            : 0
--------------------------------------------------
Missing                         : 0
--------------------------------------------------
Actual                          : 8807
--------------------------------------------------
Completeness                    : 100.0%
--------------------------------------------------
Cardinality                     : 4529
--------------------------------------------------
Uniqueness                      : 41.57%
--------------------------------------------------
Distinctness                    : 51.43%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types                : 1
--------------------------------------------------
Field Length (MIN)              : 0
--------------------------------------------------
Field Length (MAX)              : 208
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ DIRECTOR — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
                                    2634       29.91%
Rajiv Chilaka                         19        0.22%
Raúl Campos, Jan Suter                18         0.2%
Suhas Kadav                           16        0.18%
Marcus Raboy                          16        0.18%


###2.5 cast

In [ ]:
analyze_column(df, 'cast')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — CAST</span>

Field Name                      : cast
--------------------------------------------------
Field Data Type                 : STR
--------------------------------------------------
Field Format                    : 
--------------------------------------------------
Field Length                    : 771
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

NULL                            : 0
--------------------------------------------------
Missing                         : 0
--------------------------------------------------
Actual                          : 8807
--------------------------------------------------
Completeness                    : 100.0%
--------------------------------------------------
Cardinality                     : 7693
--------------------------------------------------
Uniqueness                      : 85.45%
--------------------------------------------------
Distinctness                    : 87.35%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types                : 1
--------------------------------------------------
Field Length (MIN)              : 0
--------------------------------------------------
Field Length (MAX)              : 771
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ CAST — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
                                     825        9.37%
David Attenborough                    19        0.22%
Vatsal Dubey, Julie Tejwani, R        14        0.16%
Samuel West                           10        0.11%
Jeff Dunham                            7        0.08%


###2.6 country

In [ ]:
analyze_column(df, 'country')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — COUNTRY</span>

Field Name                      : country
--------------------------------------------------
Field Data Type                 : STR
--------------------------------------------------
Field Format                    : xxxxxxxxxxxxx
--------------------------------------------------
Field Length                    : 123
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

NULL                            : 0
--------------------------------------------------
Missing                         : 0
--------------------------------------------------
Actual                          : 8807
--------------------------------------------------
Completeness                    : 100.0%
--------------------------------------------------
Cardinality                     : 749
--------------------------------------------------
Uniqueness                      : 6.39%
--------------------------------------------------
Distinctness                    : 8.5%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types                : 1
--------------------------------------------------
Field Length (MIN)              : 0
--------------------------------------------------
Field Length (MAX)              : 123
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ COUNTRY — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
United States                       2818        32.0%
India                                972       11.04%
                                     831        9.44%
United Kingdom                       419        4.76%
Japan                                245        2.78%


###2.7 date_added

In [ ]:
analyze_column(df, 'date_added')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — DATE_ADDED</span>

Field Name                      : date_added
--------------------------------------------------
Field Data Type                 : STR
--------------------------------------------------
Field Format                    : xxxxxxxxxxnnxxnnnn
--------------------------------------------------
Field Length                    : 19
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

NULL                            : 0
--------------------------------------------------
Missing                         : 0
--------------------------------------------------
Actual                          : 8807
--------------------------------------------------
Completeness                    : 100.0%
--------------------------------------------------
Cardinality                     : 1768
--------------------------------------------------
Uniqueness                      : 6.21%
--------------------------------------------------
Distinctness                    : 20.07%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types                : 1
--------------------------------------------------
Field Length (MIN)              : 0
--------------------------------------------------
Field Length (MAX)              : 19
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ DATE_ADDED — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
January 1, 2020                      109        1.24%
November 1, 2019                      89        1.01%
March 1, 2018                         75        0.85%
December 31, 2019                     74        0.84%
October 1, 2018                       71        0.81%


###2.8 release_year

In [ ]:
analyze_column(df, 'release_year')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — RELEASE_YEAR</span>

Field Name                      : release_year
--------------------------------------------------
Field Data Type                 : INT
--------------------------------------------------
Field Format                    : nnnn
--------------------------------------------------
Field Length                    : 4
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

NULL                            : 0
--------------------------------------------------
Missing                         : 0
--------------------------------------------------
Actual                          : 8807
--------------------------------------------------
Completeness                    : 100.0%
--------------------------------------------------
Cardinality                     : 74
--------------------------------------------------
Uniqueness                      : 0.06%
--------------------------------------------------
Distinctness                    : 0.84%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types                : 1
--------------------------------------------------
Field Length (MIN)              : 4
--------------------------------------------------
Field Length (MAX)              : 4
--------------------------------------------------
Field Value (MIN)               : 1925
--------------------------------------------------
Field Value (MAX)               : 2021
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ RELEASE_YEAR — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
2018                                1147       13.02%
2017                                1032       11.72%
2019                                1030        11.7%
2020                                 953       10.82%
2016                                 902       10.24%


###2.9 rating

In [ ]:
analyze_column(df, 'rating')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — RATING</span>

Field Name                      : rating
--------------------------------------------------
Field Data Type                 : STR
--------------------------------------------------
Field Format                    : xxxnn
--------------------------------------------------
Field Length                    : 8
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

NULL                            : 0
--------------------------------------------------
Missing                         : 0
--------------------------------------------------
Actual                          : 8807
--------------------------------------------------
Completeness                    : 100.0%
--------------------------------------------------
Cardinality                     : 18
--------------------------------------------------
Uniqueness                      : 0.03%
--------------------------------------------------
Distinctness                    : 0.2%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types                : 1
--------------------------------------------------
Field Length (MIN)              : 0
--------------------------------------------------
Field Length (MAX)              : 8
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ RATING — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
TV-MA                               3207       36.41%
TV-14                               2160       24.53%
TV-PG                                863         9.8%
R                                    799        9.07%
PG-13                                490        5.56%


###2.10 duration

In [ ]:
analyze_column(df, 'duration')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — DURATION</span>

Field Name                      : duration
--------------------------------------------------
Field Data Type                 : STR
--------------------------------------------------
Field Format                    : nnxxxx
--------------------------------------------------
Field Length                    : 10
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

NULL                            : 0
--------------------------------------------------
Missing                         : 0
--------------------------------------------------
Actual                          : 8807
--------------------------------------------------
Completeness                    : 100.0%
--------------------------------------------------
Cardinality                     : 221
--------------------------------------------------
Uniqueness                      : 0.37%
--------------------------------------------------
Distinctness                    : 2.51%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types                : 1
--------------------------------------------------
Field Length (MIN)              : 0
--------------------------------------------------
Field Length (MAX)              : 10
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ DURATION — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
1 Season                            1793       20.36%
2 Seasons                            425        4.83%
3 Seasons                            199        2.26%
90 min                               152        1.73%
94 min                               146        1.66%


###2.11 listed_in

In [ ]:
analyze_column(df, 'listed_in')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — LISTED_IN</span>

Field Name                      : listed_in
--------------------------------------------------
Field Data Type                 : STR
--------------------------------------------------
Field Format                    : xxxxxxxxxxxxx
--------------------------------------------------
Field Length                    : 79
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

NULL                            : 0
--------------------------------------------------
Missing                         : 0
--------------------------------------------------
Actual                          : 8807
--------------------------------------------------
Completeness                    : 100.0%
--------------------------------------------------
Cardinality                     : 514
--------------------------------------------------
Uniqueness                      : 1.61%
--------------------------------------------------
Distinctness                    : 5.84%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types                : 1
--------------------------------------------------
Field Length (MIN)              : 6
--------------------------------------------------
Field Length (MAX)              : 79
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ LISTED_IN — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
Dramas, International Movies         362        4.11%
Documentaries                        359        4.08%
Stand-Up Comedy                      334        3.79%
Comedies, Dramas, Internationa       274        3.11%
Dramas, Independent Movies, In       252        2.86%


###2.12 description

In [ ]:
analyze_column(df, 'description')

### <span style='color:#2e6cbb'>🔎 INPUT METADATA — DESCRIPTION</span>

Field Name                      : description
--------------------------------------------------
Field Data Type                 : STR
--------------------------------------------------
Field Format                    : xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
--------------------------------------------------
Field Length                    : 248
--------------------------------------------------


### <span style='color:#2e6cbb'>📊 DATA PROFILING SUMMARY STATISTICS</span>

NULL                            : 0
--------------------------------------------------
Missing                         : 0
--------------------------------------------------
Actual                          : 8807
--------------------------------------------------
Completeness                    : 100.0%
--------------------------------------------------
Cardinality                     : 8775
--------------------------------------------------
Uniqueness                      : 99.33%
--------------------------------------------------
Distinctness                    : 99.64%
--------------------------------------------------


### <span style='color:#2e6cbb'>📐 DATA PROFILING ADDITIONAL STATISTICS</span>

Field Data Types                : 1
--------------------------------------------------
Field Length (MIN)              : 61
--------------------------------------------------
Field Length (MAX)              : 248
--------------------------------------------------


### <span style='color:#2e6cbb'>⭐ DESCRIPTION — TOP 5 VALUES</span>

Value                              Count  Percentage
-------------------------------------------------------
Paranormal activity at a lush,         4        0.05%
A surly septuagenarian gets an         3        0.03%
Multiple women report their hu         3        0.03%
Challenged to compose 100 song         3        0.03%
A scheming matriarch plots to          2        0.02%


#Pre-processing Data


###1. Xử lý các giá trị missing và dữ liệu sai định dạng, sai vị trí
Đối với cột bị missing như ***director (đạo diễn), cast (diễn viên), country (quốc gia), date_added, rating, duration***, nhóm xử lý bằng cách thay các giá trị missing thành "Unknown". Riêng cột ***duration*** có 3 giá trị bị nhầm sang cột rating, nhóm chuyển data sang cột phù hợp. Ngoài ra, thuộc tính ***date_added*** cũng được định dạng lại theo dạng **dd/mm/yyyy**

In [ ]:
# Danh sách các cột cần xử lý giá trị thiếu
cols_to_clean = ['director', 'country', 'rating', 'duration', 'date_added']

# Thay thế các chuỗi rỗng hoặc chỉ có khoảng trắng bằng 'Unknown'
df[cols_to_clean] = df[cols_to_clean].replace(r'^\s*$', 'Unknown', regex=True)

# Thay thế NaN bằng 'Unknown'
df[cols_to_clean] = df[cols_to_clean].fillna('Unknown')

# Tách xử lý theo 'min' (phim) và 'season(s)' (TV Show)
min_mask = df['rating'].str.contains(r'\bmin(s)?\b', na=False, case=False)
season_mask = df['rating'].str.contains(r'\bseason(s)?\b', na=False, case=False)

# Với 'min' → chỉ chấp nhận nếu type là 'Movie'
invalid_min = min_mask & (df['type'] != 'Movie')
df.loc[invalid_min, ['rating', 'duration']] = 'Unknown'

valid_min = min_mask & (df['type'] == 'Movie')
df.loc[valid_min, 'duration'] = df.loc[valid_min, 'rating']
df.loc[valid_min, 'rating'] = 'Unknown'

# Với 'season(s)' → chỉ chấp nhận nếu type là 'TV Show'
invalid_season = season_mask & (df['type'] != 'TV Show')
df.loc[invalid_season, ['rating', 'duration']] = 'Unknown'

valid_season = season_mask & (df['type'] == 'TV Show')
df.loc[valid_season, 'duration'] = df.loc[valid_season, 'rating']
df.loc[valid_season, 'rating'] = 'Unknown'

# Chuẩn hóa định dạng ngày trong cột date_added
df['date_added'] = df['date_added'].apply(
    lambda x: pd.to_datetime(x, errors='coerce').strftime('%d/%m/%Y') if x != 'Unknown' else 'Unknown'
)


<ipython-input-18-f8a9d1be442c>:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  min_mask = df['rating'].str.contains(r'\bmin(s)?\b', na=False, case=False)
<ipython-input-18-f8a9d1be442c>:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  season_mask = df['rating'].str.contains(r'\bseason(s)?\b', na=False, case=False)


###2. Xử lý các thuộc tính đa trị
Các thuộc tính đa trị bao gồm: ***director, cast, country, list_in***, nhóm xử lý bằng cách tách các cột này thành các file với 2 thuộc tính **show_id** và **<thuộc tính đa trị cần tách>**
Ví dụ: với ***director***, ta tách thành file **movie_director**, nếu có nhiều director của cùng một phim thì tách thành nhiều dòng có chung thuộc tính ***show_id***. Làm tương tự với các thuộc tính còn lại.

In [ ]:
# Danh sách các thuộc tính đa trị
multivalued_columns = ['director', 'cast', 'country', 'listed_in']

for column in multivalued_columns:
    # Tách thành list từng giá trị, loại bỏ khoảng trắng dư
    df[column + '_list'] = df[column].apply(lambda x: [i.strip() for i in x.split(',') if i.strip() != ''])

    # Tạo bảng phụ: show_id + từng giá trị đơn lẻ
    exploded_df = df[['show_id', column + '_list']].explode(column + '_list')
    exploded_df = exploded_df.rename(columns={column + '_list': column})

    # Xuất ra file CSV, ghi đè nếu đã tồn tại
    exploded_df.to_csv(f'movie_{column}.csv', index=False)

# Xóa các cột gốc và cột _list đã tách ra khỏi df
for column in multivalued_columns:
    if column in df.columns:
        df.drop(columns=[column], inplace=True)
    if column + '_list' in df.columns:
        df.drop(columns=[column + '_list'], inplace=True)

print("✅ Đã tách và ghi file xong, đồng thời xóa các thuộc tính đã tách ra khỏi DataFrame gốc.")


✅ Đã tách và ghi file xong, đồng thời xóa các thuộc tính đã tách ra khỏi DataFrame gốc.


###3. Xử lý thuộc tính có nhiều loại giá trị khác nhau
Ở thuộc tính duration, nhóm nhận thấy cột ***duration*** có hai kiểu giá trị phụ thuộc vào thuộc tính khác (***type***)
- với phim có ***type*** là Movie: thời lượng được tính theo phút (mins)
- với phim có ***type*** là TV Show: thời lượng được tính theo phút (season)
Nhóm xử lý bằng cách tách file .csv ban đầu sau khi được xử lý các thuộc tính đa trị thành 2 file riêng biệt ***movie.csv*** và ***tv_show.csv***


In [ ]:
# Lọc theo type
df_movie = df[df['type'] == 'Movie'].copy()
df_tv = df[df['type'] == 'TV Show'].copy()

# Ghi ra file CSV
df_movie.to_csv('movie.csv', index=False)
print("✅ movie.csv saved at:", os.path.abspath('movie.csv'))

df_tv.to_csv('tv_show.csv', index=False)
print("✅ tv_show.csv saved at:", os.path.abspath('tv_show.csv'))

# Kiểm tra các file đã ghi
print("📁 Danh sách file hiện tại:")
print(os.listdir())

✅ movie.csv saved at: /content/movie.csv
✅ tv_show.csv saved at: /content/tv_show.csv
📁 Danh sách file hiện tại:
['.config', 'tv_show.csv', 'movie.csv', 'movie_cast.csv', 'netflix_titles.csv', 'movie_director.csv', 'movie_listed_in.csv', 'movie_country.csv', 'sample_data']
